In [ ]:
%%capture

%pip install numpy ollama langchain_text_splitters python-dotenv pandas psycopg2-binary Jinja2

---

If you've got a spare terminal window, you can tail the vectorizer logs
```sh
docker compose logs -f vectorizer-worker
```

Let's get our imports out of the way.

In [1]:
import json
import os.path
from os import listdir
from os.path import isfile, join
import re
from ollama import Client
import time
import numpy as np
from langchain_text_splitters import RecursiveCharacterTextSplitter #Importing this to show chunks
from dotenv import load_dotenv
from pathlib import Path
import re
from dotenv import load_dotenv
import os

import db_methods as db
import model_methods as mdl

In [2]:
content_directory = "texts"

db.create_table(content_directory)

# When you're done, you can clean up after yourself to save some space.
# drop_text_tables(content_directory)
client = Client(
    host='http://192.168.137.117:11434',
)
prompt_size=8000
chunking_size=500
chunk_count = prompt_size // chunking_size
embed_model = "nomic-embed-text"
generate_model = "gemma3:12b"
SYSTEM_PROMPT = """
You are a helpful reading assistant who answers questions
based on snippets of text provided in context. Answer only using the context provided,
being as concise as possible. If you're unsure, just say that you don't know.
Context:
"""